In [2]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!pip install datasets

In [4]:
from datasets import load_dataset

ds = load_dataset("rajpurkar/squad_v2")
dataset=ds
dataset

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 130319
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 11873
    })
})

In [5]:
from transformers import AutoTokenizer, TextDataset

# Replace with your dataset path in Google Drive
txt_path = "/content/drive/My Drive/training_data.txt"

# Function to extract conversations from a dataset and save to a text file
def extract_conversations_to_text(dataset, output_txt_path):
    with open(output_txt_path, "w", encoding="utf-8") as f:
        for example in dataset:
            user_input = example['question']
            bot_response = example['answers']
            f.write(f"User: {user_input}\nBot: {bot_response}\n")

# Extract conversations from dataset and save to a text file
extract_conversations_to_text(dataset['train'], txt_path)

# Load the tokenizer
model_name = "facebook/blenderbot-400M-distill"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Load and tokenize the dataset
train_dataset = TextDataset(
    tokenizer=tokenizer,
    file_path=txt_path,
    block_size=128,
)


/usr/local/lib/python3.10/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [6]:
!pip install transformers[torch]

In [7]:
pip install --upgrade pip setuptools


In [8]:
!pip install accelerate -U

In [2]:
from transformers import Trainer, TrainingArguments, AutoModelForCausalLM, DataCollatorForLanguageModeling

# Function to get data collator
def get_data_collator(tokenizer):
    return DataCollatorForLanguageModeling(
        tokenizer=tokenizer,
        mlm=False,
    )

# Fine-tune the model
training_args = TrainingArguments(
    output_dir="./results",
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_device_train_batch_size=7,
    save_steps=10_000,
    save_total_limit=2,
)

model = AutoModelForCausalLM.from_pretrained(model_name)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=get_data_collator(tokenizer),
    train_dataset=train_dataset,
)

trainer.train()
trainer.save_model()
tokenizer.save_pretrained("./results")


ImportError: Using the `Trainer` with `PyTorch` requires `accelerate>=0.21.0`: Please run `pip install transformers[torch]` or `pip install accelerate -U`

In [1]:
# Load the fine-tuned model
fine_tuned_model_name = "./results"
tokenizer = AutoTokenizer.from_pretrained(fine_tuned_model_name)
model = AutoModelForCausalLM.from_pretrained(fine_tuned_model_name)

# Initialize the chatbot model with the fine-tuned model
chatbot = pipeline("conversational", model=model, tokenizer=tokenizer)

# Placeholder for chat history
chat_history = []

def chat_with_bot(user_input, chat_history):
    chat_history.append(user_input)
    conversation = Conversation(user_input)
    bot_response = chatbot(conversation)
    response_text = bot_response.generated_responses[-1]
    chat_history.append(response_text)
    return response_text, chat_history

# Simulate interaction in Colab
while True:
    user_input = input("You: ")
    if user_input.lower() == 'exit':
        print("Exiting chat...")
        break
    response, chat_history = chat_with_bot(user_input, chat_history)
    print(f"Bot: {response}")
    print("\n### Chat History ###")
    for i, message in enumerate(chat_history):
        if i % 2 == 0:
            print(f"You: {message}")
        else:
            print(f"Bot: {message}")
    print("####################\n")


NameError: name 'AutoTokenizer' is not defined